In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

### 1-1. `Amazon.csv` 파일을 `df` 변수로 불러오고, 상위 5개의 행을 표시하시오.

In [ ]:
df = pd.read_csv('Amazon.csv')
df.head()

### 1-2. 아래 그림과 같이 시각화 하시오.

![nn](score_plot.png)

In [ ]:
ax = df['Score'].value_counts().plot(kind='bar', figsize=(6,6))
fig = ax.get_figure()
ax.set_title("Amazon's Fine Food Reviews Dataset")
ax.set_xlabel('Score')
ax.set_ylabel('Value Counts')

### 1-3 Score 값 중 유용하지 않은 정보는 (a) 이다.

답 : 3

### 1-4. null 값과 score 값이 (a)인 행을 제거하시오.

In [ ]:
# null 값 제거
df.dropna(inplace=True)

# score 값이 3인 값 제거
df = df[df['Score'] != 3]

### 1-5. `Positivity` 라는 컬럼을 생성하고, score 값이 (a)보다 크면 1, 작으면 0을 삽입하시오.

In [ ]:
df['Positivity'] = np.where(df['Score'] > 3, 1, 0)

In [ ]:
## 문제 X
df.head()

### 1-6. X, y 변수에 알맞는 값을 할당하시오.

In [ ]:
X = df['Text']
y = df['Positivity']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

### 1-7. countvectorizer 객체를 생성하고 학습 후, 테스트 데이터에 feature extraction 변환을 수행하시오.

In [ ]:
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)

### 1-8. LogisticRegression을 이용하여 학습/예측/평가를 수행하시오.

In [ ]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

### 1-9. 전체 문서에 걸쳐 5번 이하로 나타나는 단어는 피처로 추출하지 않고, ngram_range = (1, 2)로 설정하여 feature vectorization을 적용하시오.

In [ ]:
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

### 1-10. 최적 C값 도출 튜닝을 수행하고, 최적 C값으로 학습된 grid_cv를 예측 수행하고 정확도를 평가하시오.

In [ ]:
# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))